<h1 style="background-color: black; border: 1px solid Black;  width: 85%; padding: 30px; margin:auto; text-align: center; border-radius: 10px 0; font-weight: bold; color: white;">Estruturando os registros de acidentes da base de dados da PRF</h1>

In [1]:
# Bibliotecas a serem utilizadas
from pyspark.sql.session import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window
from pyspark.sql import types


In [2]:
spark = (
    SparkSession.builder
    .master('local')
    .appName('Projeto_Spark_PRF')
    .getOrCreate()
)

22/07/14 23:09:53 WARN Utils: Your hostname, LAPTOP-EEVQJBOG resolves to a loopback address: 127.0.1.1; using 172.17.246.73 instead (on interface eth0)
22/07/14 23:09:53 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/07/14 23:09:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [19]:
df_2017 = spark.read.csv('./PRF/datatran2017.csv', header=True, inferSchema=True, encoding='Latin1', sep=';')
df_2018 = spark.read.csv('./PRF/datatran2018.csv', header=True, inferSchema=True, encoding='Latin1', sep=';')
df_2019 = spark.read.csv('./PRF/datatran2019.csv', header=True, inferSchema=True, encoding='Latin1', sep=';')
df_2020 = spark.read.csv('./PRF/datatran2020.csv', header=True, inferSchema=True, encoding='Latin1', sep=';')
df_2021 = spark.read.csv('./PRF/datatran2021.csv', header=True, inferSchema=True, encoding='Latin1', sep=';')
df_2022 = spark.read.csv('./PRF/datatran2022.csv', header=True, inferSchema=True, encoding='Latin1', sep=';')

<h3>Juntar os DataFrames em um único:</h3>

In [20]:
df = df_2017.union(df_2018).union(df_2019).union(df_2020).union(df_2021).union(df_2022)

<h3>Verificar se existe colunas com valores nulos:</h3>

In [37]:
for coluna in df.columns:
    if df.filter(df[coluna].isNull()).count() > 0:
        print(f'Coluna:{coluna}, Quantidade de nulos:{df.filter(df[coluna].isNull()).count()}')

Coluna:uop, Quantidade de nulos:11091


<h3>Elimine as colunas desnecessárias e organize os seus tipos primitivos:</h3>

In [39]:
df.printSchema()

root
 |-- id: double (nullable = true)
 |-- data_inversa: timestamp (nullable = true)
 |-- dia_semana: string (nullable = true)
 |-- horario: timestamp (nullable = true)
 |-- uf: string (nullable = true)
 |-- br: string (nullable = true)
 |-- km: string (nullable = true)
 |-- municipio: string (nullable = true)
 |-- causa_acidente: string (nullable = true)
 |-- tipo_acidente: string (nullable = true)
 |-- classificacao_acidente: string (nullable = true)
 |-- fase_dia: string (nullable = true)
 |-- sentido_via: string (nullable = true)
 |-- condicao_metereologica: string (nullable = true)
 |-- tipo_pista: string (nullable = true)
 |-- tracado_via: string (nullable = true)
 |-- uso_solo: string (nullable = true)
 |-- pessoas: integer (nullable = true)
 |-- mortos: integer (nullable = true)
 |-- feridos_leves: integer (nullable = true)
 |-- feridos_graves: integer (nullable = true)
 |-- ilesos: integer (nullable = true)
 |-- ignorados: integer (nullable = true)
 |-- feridos: integer (null

In [42]:
temp = df
temp = temp.drop('dia_semana', 'horario', 'km', 'sentido_via', 'tipo_pista', 'tracado_via', 'uso_solo', 'latitude', 'longitude','regional','delegacia', 'uop')

In [52]:
temp = temp.withColumn('id', col('id').cast(IntegerType())).withColumn('br', col('br').cast(IntegerType()))

<h3>Salve o DataFrame em um arquivo CSV:</h3>

In [69]:
temp.coalesce(1).write.format('com.databricks.spark.csv').mode('overwrite').save('./PRF/dados_prf[2017_2022]', header=True, delimiter=';', enconding='Latin1')